In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\rcnt_class"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

In [3]:
dataset = pd.read_csv(r"C:\Users\Runker\Desktop\CNT_TRAIN\train_table\train.csv")

In [4]:
dataset.columns

Index(['Unnamed: 0', 'OBJECTID_1', 'CLASS', 'DEM_COUNT', 'DEM_AREA', 'DEM_MIN',
       'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN',
       'DEM_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA',
       'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE',
       'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM',
       'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'ProfileCurvature_COUNT',
       'ProfileCurvature_AREA', 'ProfileCurvature_MIN', 'ProfileCurvature_MAX',
       'ProfileCurvature_RANGE', 'ProfileCurvature_MEAN',
       'ProfileCurvature_STD', 'ProfileCurvature_SUM',
       'ProfileCurvature_MEDIAN', 'ProfileCurvature_PCT90', 'Slope_COUNT',
       'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN',
       'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'ndvi_COUNT',
       'ndvi_AREA', 'ndvi_MIN', 'ndvi_MAX', 'ndvi_RANGE', 'ndvi_MEAN',
       'ndvi_STD', 'ndvi_SUM', 'ndvi_MEDIAN', 'ndvi_PCT90', 

In [5]:
dataset["CLASS_label"] = dataset.CLASS.astype("category").cat.codes

In [6]:

dataset['CLASS_label'] = dataset['CLASS_label'].astype('category')

In [7]:
result = dataset.groupby('CLASS_label', observed=True)["CLASS"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: 'IS_CNT', 1: 'NOT_CNT'}


In [8]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\cnt_dict.json', 'w') as f:
    json.dump(result, f)

In [9]:
dataset.columns

Index(['Unnamed: 0', 'OBJECTID_1', 'CLASS', 'DEM_COUNT', 'DEM_AREA', 'DEM_MIN',
       'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN',
       'DEM_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA',
       'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE',
       'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM',
       'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'ProfileCurvature_COUNT',
       'ProfileCurvature_AREA', 'ProfileCurvature_MIN', 'ProfileCurvature_MAX',
       'ProfileCurvature_RANGE', 'ProfileCurvature_MEAN',
       'ProfileCurvature_STD', 'ProfileCurvature_SUM',
       'ProfileCurvature_MEDIAN', 'ProfileCurvature_PCT90', 'Slope_COUNT',
       'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN',
       'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'ndvi_COUNT',
       'ndvi_AREA', 'ndvi_MIN', 'ndvi_MAX', 'ndvi_RANGE', 'ndvi_MEAN',
       'ndvi_STD', 'ndvi_SUM', 'ndvi_MEDIAN', 'ndvi_PCT90', 

In [10]:
features= ['DEM_MIN',
       'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 
       'DEM_PCT90',
       'PlanCurvature_RANGE',
       'PlanCurvature_MEAN', 'PlanCurvature_STD', 
       'ProfileCurvature_RANGE', 'ProfileCurvature_MEAN',
       'ProfileCurvature_STD',  'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN',
       'Slope_STD',  'ndvi_MIN', 'ndvi_MAX', 'ndvi_RANGE', 'ndvi_MEAN',
       'ndvi_STD',  'PCA_0_MEAN',
       'PCA_0_STD', 'MRRTF_MEAN',
       'MRRTF_STD', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE', 'MRVBF_MEAN',
       'MRVBF_STD'
         ]

target = "CLASS_label"

In [11]:
# 获取数据集
train_data = dataset[features+[target]]

In [16]:
# 分割训练集和测试集
data_train,data_test = train_test_split(train_data, test_size=0.2, random_state=42)


# data = data
label = "CLASS_label"

In [17]:

hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


In [19]:
# 训练模型
cnt_model_path = os.path.join(model_path,'cnt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(cnt_model_path):
    os.makedirs(cnt_model_path)
else:
    print("文件夹已存在")
cnt_predictor = TabularPredictor(label=label,path=cnt_model_path,eval_metric="f1_weighted",problem_type='binary').fit(data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\rcnt_class\cnt_model"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       47.87 GB / 63.81 GB (75.0%)
Disk Space Avail:   475.25 GB / 1

文件夹已存在


	0.8103	 = Validation score   (f1_weighted)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8059	 = Validation score   (f1_weighted)
	0.63s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8002	 = Validation score   (f1_weighted)
	0.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7958	 = Validation score   (f1_weighted)
	0.4s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8003	 = Validation score   (f1_weighted)
	3.52s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.803	 = Validation score   (f1_weighted)
	8.47s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesGini': 0.571, 'RandomForestGini': 0.286, 'ExtraTreesEntr': 0.143}
	0.8151	 = Validation score   (f1_weighted)
	0.1s	 = Training   runtime
	0.0s	 = Va

In [20]:
# 获取最佳模型
best_model = cnt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
model_summary = cnt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.815124  f1_weighted       0.100678  1.497432                0.000000           0.099919            2       True          7
1     RandomForestGini   0.810328  f1_weighted       0.034642  0.582069                0.034642           0.582069            1       True          1
2     RandomForestEntr   0.805891  f1_weighted       0.031251  0.632179                0.031251           0.632179            1       True          2
3       NeuralNetTorch   0.803001  f1_weighted       0.002144  8.468794                0.002144           8.468794            1       True          6
4      NeuralNetFastAI   0.800312  f1_weighted       0.000000  3.515351                0.000000           3.515351            1       True          5
5       ExtraTreesGini   0.800213  f1_

In [21]:
cnt_predictor.leaderboard(data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.981438,0.815124,f1_weighted,0.239213,0.100678,1.497432,0.000000,0.000000,0.099919,2,True,7
1,RandomForestGini,0.980978,0.810328,f1_weighted,0.073874,0.034642,0.582069,0.073874,0.034642,0.582069,1,True,1
2,RandomForestEntr,0.980523,0.805891,f1_weighted,0.067199,0.031251,0.632179,0.067199,0.031251,0.632179,1,True,2
3,ExtraTreesGini,0.979920,0.800213,f1_weighted,0.083318,0.031943,0.417959,0.083318,0.031943,0.417959,1,True,3
4,ExtraTreesEntr,0.979467,0.795796,f1_weighted,0.082021,0.034093,0.397484,0.082021,0.034093,0.397484,1,True,4
5,NeuralNetTorch,0.835895,0.803001,f1_weighted,0.025421,0.002144,8.468794,0.025421,0.002144,8.468794,1,True,6
6,NeuralNetFastAI,0.821646,0.800312,f1_weighted,0.058874,0.000000,3.515351,0.058874,0.000000,3.515351,1,True,5


In [22]:
cnt_predictor.leaderboard(data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.816188,0.805891,f1_weighted,0.050202,0.031251,0.632179,0.050202,0.031251,0.632179,1,True,2
1,RandomForestGini,0.813104,0.810328,f1_weighted,0.058878,0.034642,0.582069,0.058878,0.034642,0.582069,1,True,1
2,ExtraTreesEntr,0.806947,0.795796,f1_weighted,0.069153,0.034093,0.397484,0.069153,0.034093,0.397484,1,True,4
3,WeightedEnsemble_L2,0.806328,0.815124,f1_weighted,0.194786,0.100678,1.497432,0.000790,0.000000,0.099919,2,True,7
4,ExtraTreesGini,0.802542,0.800213,f1_weighted,0.065965,0.031943,0.417959,0.065965,0.031943,0.417959,1,True,3
5,NeuralNetTorch,0.796644,0.803001,f1_weighted,0.015505,0.002144,8.468794,0.015505,0.002144,8.468794,1,True,6
6,NeuralNetFastAI,0.793287,0.800312,f1_weighted,0.015961,0.000000,3.515351,0.015961,0.000000,3.515351,1,True,5


In [23]:
# 获取模型特征重要性
feature_importance = cnt_predictor.feature_importance(data=data_train)
pd.DataFrame(feature_importance)

Computing feature importance via permutation shuffling for 31 features using 5000 rows with 5 shuffle sets...
	37.48s	= Expected runtime (7.5s per shuffle set)
	6.72s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
DEM_STD,0.033018,0.002347,0.000003,5,0.037852,0.028185
DEM_RANGE,0.032447,0.002835,0.000007,5,0.038285,0.026609
MRVBF_MEAN,0.005653,0.001053,0.000138,5,0.007821,0.003485
Slope_MEAN,0.004531,0.001054,0.000327,5,0.006700,0.002361
MRVBF_MAX,0.002361,0.000622,0.000528,5,0.003641,0.001080
ndvi_STD,0.002241,0.000816,0.001784,5,0.003922,0.000561
MRRTF_STD,0.002001,0.000468,0.000335,5,0.002966,0.001037
Slope_MAX,0.002000,0.000400,0.000183,5,0.002825,0.001176
ProfileCurvature_MEAN,0.001886,0.001027,0.007395,5,0.004001,-0.000229
PlanCurvature_STD,0.001720,0.000502,0.000780,5,0.002753,0.000686
